## Ejemplo de web scraping con selenium: listas desplegables y tablas
Rafa Caballero



Primero abrimos el navegador Chrome

In [1]:
modules = ["selenium","chromedriver_autoinstaller"]


import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user",  m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)  

Instalando módulos
selenium  encontrado
chromedriver_autoinstaller  encontrado
¡Terminado!


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless') # ensure GUI is off
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()


# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

##### Ejercicio 1

Cargar la página 'http://mustcalculate.com/electronics/unitconverter.php'

In [3]:
s = 'http://mustcalculate.com/electronics/unitconverter.php'
driver.get(s)


Si aparece la ventana de cookies y queremos saltarla

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
#aceptar las cookies

try:
    cookies = driver.find_element(By.CLASS_NAME, "fc-primary-button")
    cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("No se encontró el botón de aceptar cookies o ocurrió un error:", e)

Cookies aceptadas.



##### Ejercicio 2
Cargar los valores a convertir (convertir.csv) como un Dataframe pandas

In [5]:
import pandas as pd
valores = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/convertir.csv"
df = pd.read_csv(valores,encoding= 'unicode_escape')
df

,valor,origen,destino
0,10,Binary,Octal
1,100,Fahrenheit,Celsius
2,1000,Hertz,Seconds


#### Ejercicio 3
Para cada fila:

* Poner en Value el `valor`
* En la lista desplegable elegir el Origen
* Dar al botón `Calculate`
* Buscar el resultado en la tabla Results y añadirlo a una lista

Al final añadir una columna 'resultado' al dataframe y volver a grabarlo con formato csv

Sugerencia: hacerlo al principio solo para la primera fila, y cuando funcione iterar en un bucle

Para localizar los elementos mirar esta [ayuda](https://selenium-python.readthedocs.io/locating-elements.html)

Para seleccionar un elemento de un desplegable utilizar Select. [Aquí](https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python) hay un ejemplo (buscar la respuesta que utiliza Select)

In [7]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from time import sleep

# para guardar los resultados
resultados=[]
for index, r in df.iterrows():
    # meter los valores en la caja de texto
    caja = driver.find_element(By.NAME ,"in")
    caja.clear()
    caja.send_keys(str(r.valor))

    # lista desplegable
    select = Select(driver.find_element(By.NAME,'ino'))
    select.select_by_visible_text(r.origen)

    # pulsar botón
    boton = driver.find_element(By.ID,'calcbutton')
    boton.click()

    # buscar en la tabla
    div = driver.find_element(By.ID,'answer')
    trs = div.find_elements(By.XPATH,"table/tbody/tr")
    sleep(2)
    found = False
    i = 0
    while not found and i<len(trs): 
        tr = trs[i]
        # ojo con la cabecera, que tiene otro formato
        listatds = tr.find_elements(By.XPATH,"td")
        if len(listatds)>1:
            nombre = listatds[0].text
            valor = listatds[1].text
            if nombre==r.destino:
                resultados.append(valor)
                found=True
        i+=1
    
    
resultados

['2', '37.77777777778', '0.001']

In [8]:
driver.close()

In [9]:
df["resultados"] = resultados  
df.to_csv('resultados.csv', index=False)